In [50]:
%matplotlib inline
import numpy as np
import pandas as pd
import os
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [99]:
imdb_dir = '/home/viviane/Documentos/datasets/imdb/aclImdb/aclImdb'
train_dir = os.path.join(imdb_dir, 'train')
labels = []
texts = []
for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

In [100]:
test_dir = os.path.join(imdb_dir, 'test')
labels_test = []
texts_test = []
for label_type in ['neg', 'pos']:
    dir_name = os.path.join(test_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname))
            texts_test.append(f.read())
            f.close()
            if label_type == 'neg':
                labels_test.append(0)
            else:
                labels_test.append(1)

### Questão 01

In [91]:
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text)
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', ''))
    return text

In [92]:
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [93]:
stop = stopwords.words('english')

In [102]:
def preprocessing(texto):
    texto = preprocessor(texto)
    aux = [w for w in tokenizer_porter(texto) if w not in stop]
    output = ''
    output = output + aux[0]
    for i in aux[1:]:
        output = output + ' ' + i
    return output

In [103]:
for i in range(0,len(texts)):
    texts[i] = preprocessing(texts[i])
    texts_test[i] = preprocessing(texts_test[i])

In [104]:
texts[0]

'anoth exampl whi stepehn king book made movi even carri one best ruin adapt book screen premis stori revolv around fat lawyer alway diet accident kill old gypsi woman court help judg local polic chief get even though accid wa sort hi fault wa pay attent wa drive father dead gypsi woman place curs 3 main charact billi lawyer get thinner thinner day though movi kept book part ha typic king end act wa stilt felt forc went one scene next without much anyth sort like read act play king made cameo movi sort like dave barri reinforc belief author stay author leav act actor anyon thi movi wa great seen major charact movi much better'

In [101]:
texts[0]

'Just another example of why Stepehn King\'s books should not be made into movies. (Even Carrie, one of the best, is ruined in the adaptation from book to screen.) The premise of the story revolves around a fat lawyer, always on a diet, who "accidentally" kills an old gypsy woman. In court, with the help of the judge and the local police chief, he gets off, even though the accident was sort of his fault as he was not paying attention as he was driving. The father of the dead gypsy woman places a curse on the 3, with our main character, Billy (the lawyer) getting thinner and thinner by the day. Though the movie kept with the book, for the most part, and has your typical King ending, the acting was stilted and felt forced. We went from one scene to the next without much of anything in between, sort of like reading acts in a play. King himself made a cameo in the movie (sort of like Dave Barry), which reinforces my belief that authors should stay just that: authors. Leave the acting to ac

### Questão 02

### Questão 03

In [36]:
stop = stopwords.words('english')
def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                       text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) \
       + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

In [37]:
vect = HashingVectorizer(decode_error='ignore',
   n_features=2**21, #The number of features (columns) in the output matrices.
   tokenizer=tokenizer)

In [45]:
X_train = vect.transform(texts)
X_test = vect.transform(texts_test)